In [ ]:
!pip install pgmpy

In [52]:
import pandas as pd

In [53]:
from google.colab import files
uploaded = files.upload()

Saving csv_result-caesarian.csv to csv_result-caesarian (1).csv


In [54]:
import io
cols = ["Age", "Delivery Number", "Delivery Time", "Blood Pressure", "Heart problem", "Caesarian"]
df = pd.read_csv(io.BytesIO(uploaded['csv_result-caesarian.csv']), names = cols)
data = df.iloc[1:]

In [55]:
data.head()

,Age,Delivery Number,Delivery Time,Blood Pressure,Heart problem,Caesarian
1,22,1,0,2,0,0
2,26,2,0,1,0,1
3,26,2,1,1,0,0
4,28,1,0,2,0,0
5,22,2,0,1,0,1


In [56]:
data.Age = data.Age.astype(str).astype(int)
data["Delivery Number"] = data["Delivery Number"].astype(str).astype(int)
data["Delivery Time"] = data["Delivery Time"].astype(str).astype(int)
data["Blood Pressure"] = data["Blood Pressure"].astype(str).astype(int)
data["Heart problem"] = data["Heart problem"].astype(str).astype(int)
data["Caesarian"] = data["Caesarian"].astype(str).astype(int)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [57]:
data.dtypes

Age                int64
Delivery Number    int64
Delivery Time      int64
Blood Pressure     int64
Heart problem      int64
Caesarian          int64
dtype: object

In [59]:
data.corr()

,Age,Delivery Number,Delivery Time,Blood Pressure,Heart problem,Caesarian
Age,1.000000,0.427160,-0.021857,0.074448,0.250485,0.077966
Delivery Number,0.427160,1.000000,-0.074017,0.134315,0.200267,0.144894
Delivery Time,-0.021857,-0.074017,1.000000,-0.087298,-0.003985,-0.166233
Blood Pressure,0.074448,0.134315,-0.087298,1.000000,0.036515,-0.035760
Heart problem,0.250485,0.200267,-0.003985,0.036515,1.000000,0.352557
Caesarian,0.077966,0.144894,-0.166233,-0.035760,0.352557,1.000000


In [60]:
from pgmpy.models import BayesianModel

In [87]:
model = BayesianModel([("Age", "Caesarian"),("Delivery Number", "Caesarian"),("Delivery Time", "Caesarian"),("Blood Pressure", "Caesarian"),("Heart problem", "Caesarian")])
model.fit(data)

In [88]:
from pgmpy.inference import VariableElimination
infer = VariableElimination(model)

In [90]:
condition1 = infer.query(variables=["Caesarian"],evidence={"Age":22, "Delivery Number" : 1, "Delivery Time" : 2})
print(condition1)

Eliminating: Blood Pressure: 100%|██████████| 2/2 [00:00<00:00, 438.62it/s]

+--------------+------------------+
| Caesarian    |   phi(Caesarian) |
+==============+==================+
| Caesarian(0) |           0.5781 |
+--------------+------------------+
| Caesarian(1) |           0.4219 |
+--------------+------------------+


In [91]:
condition2 = infer.query(variables=["Caesarian"],evidence={"Age" : 26, "Delivery Number" : 2, "Delivery Time" : 0, "Blood Pressure" : 1, "Blood Pressure" : 2, "Heart problem" : 0})
print(condition2)

Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]

+--------------+------------------+
| Caesarian    |   phi(Caesarian) |
+==============+==================+
| Caesarian(0) |           0.5000 |
+--------------+------------------+
| Caesarian(1) |           0.5000 |
+--------------+------------------+
